<a href="https://colab.research.google.com/github/RanjithSunder/NASA-Asteroids-Explorer/blob/main/NASA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector==2.2.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 72.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp311-cp311-linux_x86_64.whl size=247951 sha256=d12dd8d5ed17541efa5ec26cae23318a8f2ffadc6b3e065817cfc54b4397b8f6
  Stored in directory: /root/.cache/pip/wheels/17/cd/ed/2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql-connector


In [ ]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import time
import mysql.connector

In [ ]:
API_KEY = "YFIweBiyHPfMZlH2fuXemnidjfnn2GcM9ob7kpVL"
URL = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

In [ ]:
asteroids_data = []
target = 10000

In [ ]:
while len(asteroids_data) < target:
      response = requests.get(URL)
      data = response.json()
      details = data['near_earth_objects']
      for date,ast_details in details.items():
          for ast in ast_details:
            asteroids_data.append(dict(
                id = int(ast['id']), # convert format accordingly
                neo_id = int(ast['neo_reference_id']),
                name = ast['name'],
                abs_magnitude = ast['absolute_magnitude_h'],
                dia_min = ast['estimated_diameter']['kilometers']['estimated_diameter_min'],
                dia_max = ast['estimated_diameter']['kilometers']['estimated_diameter_max'],
                is_potentially_hazardous_asteroid = ast['is_potentially_hazardous_asteroid'],
                close_approach_date = datetime.datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], '%Y-%m-%d').date(),
                relative_velocity = ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour'],
                astronomical = ast['close_approach_data'][0]['miss_distance']['astronomical'],
                miss_distance_lunar = ast['close_approach_data'][0]['miss_distance']['lunar'],
                miss_distance = ast['close_approach_data'][0]['miss_distance']['kilometers'],
                orbiting_body = ast['close_approach_data'][0]['orbiting_body']
            ))

            if len(asteroids_data) == target:
                break
          if len(asteroids_data) == target:
                break
      URL = data['links']['next']


In [ ]:
print(json.dumps(ast_details,indent=2))

[
  {
    "links": {
      "self": "http://api.nasa.gov/neo/rest/v1/neo/2136818?api_key=YFIweBiyHPfMZlH2fuXemnidjfnn2GcM9ob7kpVL"
    },
    "id": "2136818",
    "neo_reference_id": "2136818",
    "name": "136818 Selqet (1997 MW1)",
    "nasa_jpl_url": "https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2136818",
    "absolute_magnitude_h": 18.98,
    "estimated_diameter": {
      "kilometers": {
        "estimated_diameter_min": 0.425162524,
        "estimated_diameter_max": 0.9506923052
      },
      "meters": {
        "estimated_diameter_min": 425.1625240058,
        "estimated_diameter_max": 950.6923051623
      },
      "miles": {
        "estimated_diameter_min": 0.2641836627,
        "estimated_diameter_max": 0.5907326284
      },
      "feet": {
        "estimated_diameter_min": 1394.8902152591,
        "estimated_diameter_max": 3119.0693424687
      }
    },
    "is_potentially_hazardous_asteroid": false,
    "close_approach_data": [
      {
        "close_approach_date"

In [ ]:
asteroids_data[0]

{'id': 3023012,
 'neo_id': 3023012,
 'name': '(1999 RA32)',
 'abs_magnitude': 21.39,
 'dia_min': 0.1401376972,
 'dia_max': 0.3133574171,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': datetime.date(2024, 2, 7),
 'relative_velocity': '27762.7959410997',
 'astronomical': '0.1494049921',
 'miss_distance_lunar': '58.1185419269',
 'miss_distance': '22350668.585526827',
 'orbiting_body': 'Earth'}

In [ ]:
conn = mysql.connector.connect(
    host='104.28.193.171',
    user='vmfg',
    password='vmfgpwd!',
    database='nasa'
)
print(conn)
cursor = conn.cursor()

for asteroid in asteroids_data:
    # Insert into asteroids table
    cursor.execute("""
        INSERT INTO asteroids (
            id,
            name,
            absolute_magnitude_h,
            estimated_diameter_min_km,
            estimated_diameter_max_km,
            is_potentially_hazardous_asteroid
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        asteroid['id'],
        asteroid['name'],
        asteroid['abs_magnitude'],
        asteroid['dia_min'],
        asteroid['dia_max'],
        asteroid['is_potentially_hazardous_asteroid']
    ))

    cursor.execute("""
        INSERT INTO close_approaches (
            neo_reference_id,
            close_approach_date,
            relative_velocity_kmph,
            astronomical_au,
            miss_distance_km,
            miss_distance_lunar,
            orbiting_body
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        asteroid['neo_id'],
        asteroid['close_approach_date'],
        float(asteroid['relative_velocity']),
        float(asteroid['astronomical']),
        float(asteroid['miss_distance']),
        float(asteroid['miss_distance_lunar']),
        asteroid['orbiting_body']
    ))

conn.commit()
cursor.close()
conn.close()